### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,roc_curve, precision_recall_curve
import dill
dill._dill._reverse_typemap['ClassType'] = type

In [2]:
X_test = pd.read_csv("data/X_test.csv")
y_test = pd.read_csv("data/y_test.csv")

In [3]:
X_test.head(3)

,Wife_age,Wife_education,Husband_education,Children,Religion,Working,Husband_Work,Standard-of-living,Media_exposure
0,29,3,3,4,1,1,3,4,0
1,35,3,4,5,1,0,2,2,0
2,23,4,4,1,0,1,1,4,0


In [4]:
with open("catboost_pipeline.dill", 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [5]:
predictions = pipeline.predict_proba(X_test)[:, 1]


In [6]:
pd.DataFrame({'preds': predictions}).to_csv("data/test_predictions.csv", index=None)

In [7]:
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

f_score = (2*precision*recall)/(precision+recall)

ix = np.argmax(f_score)

roc = roc_auc_score(y_test, predictions)

print(f'Best threshold: {thresholds[ix]}, F-score: {f_score[ix]}, precision: {precision[ix]}, recall: {recall[ix]}, roc_auc = {roc}')

Best threshold: 0.34131486006052486, F-score: 0.6932773109243698, precision: 0.6111111111111112, recall: 0.8009708737864077, roc_auc = 0.7959437515115919
